In [75]:
pip install gym==0.10.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
!git clone https://github.com/openai/multiagent-particle-envs.git

Cloning into 'multiagent-particle-envs'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 242 (delta 0), reused 3 (delta 0), pack-reused 237
Receiving objects: 100% (242/242), 107.24 KiB | 3.15 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [77]:
cd multiagent-particle-envs

/content/multiagent-particle-envs/multiagent-particle-envs


In [78]:
pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/multiagent-particle-envs/multiagent-particle-envs
  Attempting uninstall: multiagent
    Found existing installation: multiagent 0.0.1
    Can't uninstall 'multiagent'. No files were found to uninstall.
  Running setup.py develop for multiagent


In [79]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as f
import cv2
import numpy as np
from make_env import make_env
env = make_env("simple_spread")
import csv

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim


#Actor
class Actor1(nn.Module):
  def __init__(self,state_size,action_size):
    super(Actor1,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.fc1 = nn.Linear(self.state_size,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,512)
    self.fc5 = nn.Linear(512,128)
    self.fc6 = nn.Linear(128,64)
    self.fc7 = nn.Linear(64,32)
    self.fc8 = nn.Linear(32,self.action_size)
  def forward(self,x):
    x = self.fc1(x)
    x = f.relu(self.fc7(f.relu(self.fc6(f.relu(self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(self.fc2(x))))))))))))
    x = f.softmax(self.fc8(x))
    return x

class Actor2(nn.Module):
  def __init__(self,state_size,action_size):
    super(Actor2,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.fc1 = nn.Linear(self.state_size,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,512)
    self.fc5 = nn.Linear(512,128)
    self.fc6 = nn.Linear(128,64)
    self.fc7 = nn.Linear(64,32)
    self.fc8 = nn.Linear(32,self.action_size)
  def forward(self,x):
    x = self.fc1(x)
    x = f.relu(self.fc7(f.relu(self.fc6(f.relu(self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(self.fc2(x))))))))))))
    x = f.softmax(self.fc8(x))
    return x

class Actor3(nn.Module):
  def __init__(self,state_size,action_size):
    super(Actor3,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.fc1 = nn.Linear(self.state_size,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,512)
    self.fc5 = nn.Linear(512,128)
    self.fc6 = nn.Linear(128,64)
    self.fc7 = nn.Linear(64,32)
    self.fc8 = nn.Linear(32,self.action_size)
  def forward(self,x):
    x = self.fc1(x)
    x = f.relu(self.fc7(f.relu(self.fc6(f.relu(self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(self.fc2(x))))))))))))
    x = f.softmax(self.fc8(x))
    return x

In [81]:
class Critic(nn.Module):
  def __init__(self,state_size,action_size):
    super(Critic,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.fc1 = nn.Linear(self.state_size,32)
    self.fc1a= nn.Linear(self.action_size,32)
    self.fc2 = nn.Linear(64,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,512)
    self.fc5 = nn.Linear(512,128)
    self.fc6 = nn.Linear(128,64)
    self.fc7 = nn.Linear(64,32)
    self.fc8 = nn.Linear(32,1)
  def forward(self,s,a):
    s = self.fc1(s )
    a = self.fc1a(a)
    x = torch.reshape(torch.cat([s.view(32,-1),a.view(32,-1)],dim = -1),(-1,))
    x = f.relu(self.fc7(f.relu(self.fc6(f.relu(self.fc5(f.relu(self.fc4(f.relu(self.fc3(f.relu(self.fc2(x))))))))))))
    x = f.relu(self.fc8(x))
    return x

In [82]:
class Buffer:
  def __init__(self):
    self.episodes       = []
    self.message        = []
    self.agent1_reward  = []
    self.agent2_reward  = []
    self.agent3_reward  = []
    self.agent1_mean    = []
    self.agent2_mean    = []
    self.agent3_mean    = []
    self.agent1_return  = []
    self.agent2_return  = []
    self.agent3_return  = []
    self.agent1_loss    = []
    self.agent2_loss    = []
    self.agent3_loss    = []
    self.agent1_gradient= []
    self.agent2_gradient= []
    self.agent3_gradient= []
    self.agent1_state   = []
    self.agent2_state   = []
    self.agent3_state   = []
    self.agent1_next_s  = []
    self.agent2_next_s  = []
    self.agent3_next_s  = []
    self.agent1_q_value = []
    self.agent2_q_value = []
    self.agent3_q_value = []
    self.agent1_n_qvalue= []
    self.agent2_n_qvalue= []
    self.agent3_n_qvalue= []
    self.agent1_parametrs=[]
    self.agent2_parametrs=[]
    self.agent3_parametrs=[]
    self.centeral_loss   =[]
    self.centeral_returns=[]
    self.agent1_loss     =[]
    self.agent2_loss     =[]
    self.agent3_loss     =[]

In [83]:
class Agent1:
  def __init__(self,state_size,action_size,reward_size):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size
    self.lr1         = 0.000007
    self.lr2         = 0.000010
    self.gamma       = 0.99
    self.lamda       = 0.95
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor       = Actor1(self.state_size,self.action_size).to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
  def choose_actions(self,state):
    act = self.actor(state).to(self.device)
    return act
  def learn(self,state,next_state,reward,done,value,next_value):
    state = torch.from_numpy(state).float().to(self.device)
    next_state = torch.from_numpy(next_state).float().to(self.device)
    reward = torch.tensor(reward,dtype = torch.float32)
    action = self.choose_actions(state)
    baseline = torch.sum(value * action)
    advantage= torch.mean(value - baseline)
    log_prob = torch.log(action)
    loss = torch.mean(advantage * log_prob)
    buffer.agent1_loss.append(loss)
    buffer.agent1_parametrs.append(self.actor.parameters())
    self.actor_optim.zero_grad()
    loss.backward(retain_graph=True)
    self.actor_optim.step()

In [84]:
class Agent2:
  def __init__(self,state_size,action_size,reward_size):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size
    self.lr1         = 0.000007
    self.lr2         = 0.000010
    self.gamma       = 0.99
    self.lamda       = 0.95
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor       = Actor2(self.state_size,self.action_size).to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
  def choose_actions(self,state):
    act = self.actor(state).to(self.device)
    return act
  def learn(self,state,next_state,reward,done,value,next_value):
    state = torch.from_numpy(state).float().to(self.device)
    next_state = torch.from_numpy(next_state).float().to(self.device)
    reward = torch.tensor(reward,dtype = torch.float32)
    action = self.choose_actions(state)
    baseline = torch.sum(value * action)
    advantage= torch.mean(value - baseline)
    log_prob = torch.log(action)
    loss = torch.mean(advantage * log_prob)
    buffer.agent2_loss.append(loss)
    buffer.agent2_parametrs.append(self.actor.parameters())
    self.actor_optim.zero_grad()
    loss.backward(retain_graph=True)
    self.actor_optim.step()

In [85]:
class Agent3:
  def __init__(self,state_size,action_size,reward_size):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size
    self.lr1         = 0.000007
    self.lr2         = 0.000010
    self.gamma       = 0.99
    self.lamda       = 0.95
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor       = Actor3(self.state_size,self.action_size).to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
  def choose_actions(self,state):
    act = self.actor(state).to(self.device)
    return act
  def learn(self,state,next_state,reward,done,value,next_value):
    state = torch.from_numpy(state).float().to(self.device)
    next_state = torch.from_numpy(next_state).float().to(self.device)
    reward = torch.tensor(reward,dtype = torch.float32)
    action = self.choose_actions(state)
    baseline = torch.sum(value * action)
    advantage= torch.mean(value - baseline)
    log_prob = torch.log(action)
    loss = torch.mean(advantage * log_prob)
    buffer.agent3_loss.append(loss)
    buffer.agent3_parametrs.append(self.actor.parameters())
    self.actor_optim.zero_grad()
    loss.backward(retain_graph=True)
    self.actor_optim.step()

In [86]:
class Temp:
  def __init__(self):
    self.a1 = []
    self.a2 = []
    self.a3 = []

In [92]:
temp = Temp()
buffer = Buffer()

In [95]:

@torch.no_grad()
class Main:
  def __init__(self,state_size,action_size,reward_size,n_games ,step):
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size
    self.gamma       = 0.99
    self.n_games     = n_games
    self.step        = step
    self.agent1      = Agent1(self.state_size,self.action_size,self.reward_size)
    self.agent2      = Agent2(self.state_size,self.action_size,self.reward_size)
    self.agent3      = Agent3(self.state_size,self.action_size,self.reward_size)
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.critic      = Critic(self.state_size,self.action_size).to(self.device)
    self.critic_optim= optim.Adam(self.critic.parameters() ,lr = 0.0000007)
  def choose_action(self,state):
    actions = []
    obs1  = torch.from_numpy(state[0]).float().to(self.device)
    obs2  = torch.from_numpy(state[1]).float().to(self.device)
    obs3  = torch.from_numpy(state[2]).float().to(self.device)
    act1  = self.agent1.choose_actions(obs1).to(self.device)
    act2  = self.agent2.choose_actions(obs2).to(self.device)
    act3  = self.agent3.choose_actions(obs3).to(self.device)
    actions.append(act1)
    actions.append(act2)
    actions.append(act3)
    return actions
  def appending(self,state,next_state,reward,done,episode,q_value1,q_value2,q_value3,next_value1,next_value2,next_value3,loss):
    buffer.agent1_state.append(state[0])
    buffer.agent2_state.append(state[1])
    buffer.agent3_state.append(state[2])
    buffer.agent1_next_s.append(next_state[0])
    buffer.agent2_next_s.append(next_state[1])
    buffer.agent3_next_s.append(next_state[2])
    #buffer.agent1_reward.append(reward[0])
    #buffer.agent2_reward.append(reward[1])
    #buffer.agent3_reward.append(reward[2])
    buffer.agent1_q_value.append(q_value1)
    buffer.agent2_q_value.append(q_value2)
    buffer.agent3_q_value.append(q_value3)
    buffer.agent1_n_qvalue.append(next_value1)
    buffer.agent2_n_qvalue.append(next_value2)
    buffer.agent3_n_qvalue.append(next_value3)
    buffer.episodes.append(episode)
    buffer.centeral_loss.append(loss)
  def save(self,i):
    st1 = ["slno","agent1_reward","agent2_reward","agent3_reward","agent1_q_value","agent2_q_value","agent3_q_value","agent1_n_value","agent2_n_value","agent3_n_value","agent1_loss","agent2_loss","agent3_loss","agent1_paramaters","agent2_paramaters","agent3_paramaters"]
    with open("coma_parameters.csv","w") as f:
      write = csv.writer(f)
      write.writerow(st1)
      write.writerow([buffer.agent1_reward[i],buffer.agent2_reward[i],buffer.agent3_reward[i],buffer.agent1_q_value[i],buffer.agent2_q_value[i],buffer.agent3_q_value[i],buffer.agent1_n_qvalue[i],buffer.agent2_n_qvalue[i],buffer.agent3_n_qvalue[i],buffer.agent1_loss[i],buffer.agent2_loss[i],buffer.agent3_loss[i],buffer.agent1_parametrs[i],buffer.agent2_parametrs[i],buffer.agent3_parametrs[i]])
  def learn(self,state,next_state,reward,done,info,i):
    torch.autograd.set_detect_anomaly(True)
    actions = self.choose_action(state)
    next_actions = self.choose_action(next_state)
    q_value1 = self.critic(torch.from_numpy(state[0]).float().to(self.device),actions[0])
    q_value2 = self.critic(torch.from_numpy(state[1]).float().to(self.device),actions[1])
    q_value3 = self.critic(torch.from_numpy(state[2]).float().to(self.device),actions[2])
    next_q_value1 = self.critic(torch.from_numpy(next_state[0]).float().to(self.device),next_actions[0])
    next_q_value2 = self.critic(torch.from_numpy(next_state[1]).float().to(self.device),next_actions[1])
    next_q_value3 = self.critic(torch.from_numpy(next_state[2]).float().to(self.device),next_actions[2])
    q_value       = q_value1+q_value2 +q_value3
    next_value    = next_q_value1 + next_q_value2 + next_q_value3
    self.agent1.learn(state[0],next_state[0],reward[0],done[0],q_value1,next_q_value1)
    self.agent2.learn(state[1],next_state[1],reward[1],done[1],q_value2,next_q_value2)
    self.agent3.learn(state[2],next_state[2],reward[2],done[2],q_value3,next_q_value3)
    returns = sum(reward) + self.gamma*next_value - q_value
    loss    = (returns - q_value).pow(2)
    self.appending(state,next_state,reward,done,i,q_value1,q_value2,q_value3,next_q_value1,next_q_value2,next_q_value3,loss)
    self.save(i)
    #returns without trajectories
    #self.critic_optim.zero_grad()
    #loss.backward(retain_graph=True)
    #self.critic_optim.step()

  def play(self):
    for i in range(self.n_games):
      state = env.reset()
      score = [0*3]
      done  = [False * 3]
      if i==0:
        buffer.agent1_reward.append(sum(temp.a1))
        buffer.agent2_reward.append(sum(temp.a2))
        buffer.agent3_reward.append(sum(temp.a3))       
      else:
        buffer.agent1_reward.append(sum(temp.a1)/len(temp.a1))
        buffer.agent2_reward.append(sum(temp.a2)/len(temp.a1))
        buffer.agent3_reward.append(sum(temp.a3)/len(temp.a1)) 
      temp
      for step in range(self.step):
        actions = self.choose_action(state)
        next_state,reward,done,info = env.step(actions)
        if done:
          self.learn(state,next_state,reward,done,info,i)
          state = next_state
          score += reward
          print("episode:",i,"reward:",sum(reward))
          temp.a1.append(reward[0])
          temp.a2.append(reward[1])
          temp.a3.append(reward[2])
        else:
          self.learn(state,next_state,reward,done,info,i)
          state = next_state
          score += reward
          print("episode:",i,"reward:",sum(reward))
          temp.a1.append(reward[0])
          temp.a2.append(reward[1])
          temp.a3.append(reward[2])

In [ ]:
if __name__ == "__main__":
  m = Main(18,5,2,2000,300)
  m.play()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


episode: 0 reward: -20.95803452182613
episode: 0 reward: -20.95348033601907
episode: 0 reward: -20.948780809783607
episode: 0 reward: -20.945230256625674
episode: 0 reward: -20.943942089256012
episode: 0 reward: -20.945854074234276
episode: 0 reward: -20.951749544127885
episode: 0 reward: -20.9622807894269
episode: 0 reward: -20.977988930035995
episode: 0 reward: -20.999317554048016
episode: 0 reward: -21.026621319205134
episode: 0 reward: -21.060170187805312
episode: 0 reward: -21.100151582089286
episode: 0 reward: -21.146671677213906
episode: 0 reward: -21.19975733737465
episode: 0 reward: -21.259360315945784
episode: 0 reward: -21.32536346717754
episode: 0 reward: -21.39758919060425
episode: 0 reward: -21.475810054172275
episode: 0 reward: -21.559760021854515
episode: 0 reward: -21.649145779112175
episode: 0 reward: -21.7436574845636
episode: 0 reward: -21.842978595018266
episode: 0 reward: -21.94679400446342
episode: 0 reward: -22.054796371993834
episode: 0 reward: -22.166690821983

In [ ]:
def appending(self,state,next_state,reward,done,episode,q_value1,q_value2,q_value3,next_value1,next_value2,next_value3,loss):
  buffer.agent1_state.append(state[0])
  buffer.agent2_state.append(state[1])
  buffer.agent3_state.append(state[2])
  buffer.agent1_next_s.append(next_state[0])
  buffer.agent2_next_s.append(next_state[1])
  buffer.agent3_next_s.append(next_state[2])
  buffer.agent1_reward.append(reward[0])
  buffer.agent2_reward.append(reward[1])
  buffer.agent3_reward.append(reward[2])
  buffer.agent1_q_value.append(q_value1)
  buffer.agent2_q_value.append(q_value2)
  buffer.agent3_q_value.append(q_value3)
  buffer.agent1_n_qvalue.append(next_value1)
  buffer.agent2_n_qvalue.append(next_value2)
  buffer.agent3_n_qvalue.append(next_value3)
  buffer.episodes.append(episode)
  buffer.centeral_loss.append(loss)

In [ ]:
def save(i):
  st1 = ["slno","agent1_reward","agent2_reward","agent3_reward","agent1_q_value","agent2_q_value","agent3_q_value","agent1_n_value","agent2_n_value","agent3_n_value","agent1_loss","agent2_loss","agent3_loss","agent1_paramaters","agent2_paramaters","agent3_paramaters"]
  with open("coma_parameters.csv","w") as f:
    write = csv.writer(f)
    write.writerow(st1)
    write.writerow([buffer.agent1_reward[i],buffer.agent2_reward[i],buffer.agent3_reward[i],buffer.agent1_q_value[i],buffer.agent2_q_value[i],buffer.agent3_q_value[i],buffer.agent1_n_qvalue[i],buffer.agent2_n_qvalue[i],buffer.agent3_n_qvalue[i],buffer.agent1_loss[i],buffer.agent2_loss[i],buffer.agent3_loss[i],buffer.agent1_parametrs[i],buffer.agent2_parametrs[i],buffer.agent3_parametrs[i]])